# Assess income level predictions on adult census data

This notebook demonstrates the use of the `responsibleai` API to assess a model trained on census data. It walks through the API calls necessary to create a widget with model analysis insights, then guides a visual analysis of the model.

* [Launch Responsible AI Toolbox](#Launch-Responsible-AI-Toolbox)
    * [Train a Model](#Train-a-Model)
    * [Create Model and Data Insights](#Create-Model-and-Data-Insights)
* [Assess Your Model](#Assess-Your-Model)
    * [Aggregate Analysis](#Aggregate-Analysis)
    * [Individual Analysis](#Individual-Analysis)

## Launch Responsible AI Toolbox

The following section examines the code necessary to create datasets and a model. It then generates insights using the `responsibleai` API that can be visually analyzed.

### Train a Model
*The following section can be skipped. It loads a dataset and trains a model for illustrative purposes.*

In [1]:
import sklearn
from typing import Any, Callable, Dict, List, Optional, Set, Tuple
import zipfile

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

import pandas as pd
from lightgbm import LGBMClassifier

First, load the census dataset and specify the different types of features. Then, clean the target feature values to include only 0 and 1.

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def split_label(dataset, target_feature):
    X = dataset.drop([target_feature], axis=1)
    y = dataset[[target_feature]]
    return X, y

def clean_data(X, y, target_feature):
    features = X.columns.values.tolist()
    classes = y[target_feature].unique().tolist()
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = feat_pipe.fit_transform(X)
    print(pipe_cfg['cat_cols'])
    return X, feat_pipe, features, classes

outdirname = 'responsibleai.12.28.21'
try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve
zipfilename = outdirname + '.zip'
urlretrieve('https://publictestdatasets.blob.core.windows.net/data/' + zipfilename, zipfilename)
with zipfile.ZipFile(zipfilename, 'r') as unzip:
    unzip.extractall('.')

target_feature = 'income'
categorical_features = ['workclass', 'education', 'marital-status',
                        'occupation', 'relationship', 'race', 'gender', 'native-country']


train_data = pd.read_csv('adult-train.csv')
test_data = pd.read_csv('adult-test.csv')


X_train_original, y_train = split_label(train_data, target_feature)
X_test_original, y_test = split_label(test_data, target_feature)


X_train, feat_pipe, features, classes = clean_data(X_train_original, y_train, target_feature)
y_train = y_train[target_feature].to_numpy()

X_test = feat_pipe.transform(X_test_original)
y_test = y_test[target_feature].to_numpy()

train_data[target_feature] = y_train
test_data[target_feature] = y_test

test_data_sample = test_data.sample(n=500, random_state=5)
train_data_sample = train_data.sample(n=8000, random_state=5)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']


Train a LightGBM classifier on the training data.

In [3]:
clf = LGBMClassifier(n_estimators=5)
model = clf.fit(X_train, y_train)

In [4]:
dashboard_pipeline = Pipeline(steps=[('preprocess', feat_pipe), ('model', model)])

## Visualize Data Balance Measures

### Option #1 - Let RAI compute your measures

In [5]:
train_data

,income,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,<=50K,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,<=50K,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,<=50K,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,<=50K,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,<=50K,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,<=50K,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,>50K,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,<=50K,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,<=50K,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [6]:
from responsibleai import DataBalanceAnalysis, RAIInsights
from raiwidgets import ResponsibleAIDashboard

# Pass in information about our data so that RAI can properly compute data balance measures
data_balance_analysis = DataBalanceAnalysis(
    df=train_data,
    cols_of_interest=["race", "gender"],  # columns to use for data balance analysis. Future plan: let RAI suggest potential columns to the user
    label_col="income",
    label_func=lambda x: 0 if x == "<=50K" else 1  # convert {"<=50k", ">50k"} to {0, 1} respectively
)

rai_insights = RAIInsights(
    # Params that were already here:
    model=dashboard_pipeline,
    train=train_data_sample,
    test=test_data_sample,
    target_column=target_feature,
    task_type="classification",
    categorical_features=categorical_features,
    # Added params:
    dataset_name="Adult Census",
    data_balance_analysis=data_balance_analysis,
)

ResponsibleAIDashboard(rai_insights)

C:\Users\kashy\anaconda3\envs\rai\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


ResponsibleAI started at http://localhost:5000


In [7]:
# In the code, we create a deep copy of train_data before transforming it/computing measures so the original data is not modified
train_data

,income,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,<=50K,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,<=50K,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,<=50K,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,<=50K,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,<=50K,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,<=50K,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,>50K,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,<=50K,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,<=50K,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


### Data Balance Measures: Option #2 - Pass in your own measures

In [8]:
df = pd.read_csv("adult-train.csv")

# convert {"<=50k", ">50k"} to {0, 1} respectively 
df["income"] = df["income"].apply(lambda x: 0 if x == "<=50K" else 1)

cols_of_interest = ["race", "gender"]
label_col = "income"

df

,income,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,0,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,0,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,0,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,1,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,0,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,0,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [9]:
from databalanceanalysis.feature_measures import FeatureBalanceMeasure
from databalanceanalysis.constants import Measures

# Compute measures
feature_measures_df = FeatureBalanceMeasure(cols_of_interest, label_col).measures(df)

# Transform df into into a dictionary acceptable by the RAI dashboard
rows: List[Any] = feature_measures_df.reset_index(drop=True).to_dict(orient="records")
unique_values: Dict[str, Set[str]] = {
    r["feature_name"]: set() for r in rows
}
feature_measures: Dict[str, Dict[str, Dict[str, float]]] = {
    r["feature_name"]: {} for r in rows
}

for row in rows:
    feature_name: str = row["feature_name"]
    class_a: str = row["classA"]
    class_b: str = row["classB"]
    class_key: str = f"{class_a}__{class_b}"

    unique_values[feature_name].add(class_a)
    unique_values[feature_name].add(class_b)

    del row["feature_name"]
    del row["classA"]
    del row["classB"]

    # TODO: This won't be necessary once cols are converted to strings
    row: Dict[str, float] = {k.value: v for k, v in row.items()}

    feature_measures[feature_name][class_key] = row

unique_values: Dict[str, List[str]] = {
    k: list(v) for k, v in unique_values.items()
}

feature_measures_df.sort_values(by=Measures.DEMOGRAPHIC_PARITY, key=abs, ascending=False)

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Male,Female,gender,0.196276,1.027159,0.161486,0.222413,-0.880102,1.731753,-0.372541,0.386486
8,Asian-Pac-Islander,Other,race,0.173389,1.057630,0.027999,0.028987,-6.580105,2.401525,-2.674763,0.027765
3,White,Other,race,0.163609,1.020118,0.196514,0.246278,-15.322864,5.651366,-5.256855,0.003134
7,Asian-Pac-Islander,Amer-Indian-Eskimo,race,0.149884,0.830661,0.026665,0.027642,-5.690376,2.036882,-2.313755,0.040847
4,Black,Asian-Pac-Islander,race,-0.141760,-0.762832,0.004213,0.004507,-3.330252,0.338024,-1.227189,0.143519
2,White,Amer-Indian-Eskimo,race,0.140104,0.793149,0.195180,0.244934,-14.433135,5.286723,-4.895847,0.004034
0,White,Black,race,0.131980,0.725320,0.164302,0.212784,-5.412507,2.911817,-1.354903,0.123458
6,Black,Other,race,0.031629,0.294798,0.032212,0.033494,-9.910357,2.739549,-3.901952,0.008756
9,Amer-Indian-Eskimo,Other,race,0.023505,0.226969,0.001334,0.001344,-0.889729,0.364643,-0.361008,0.368176
1,White,Asian-Pac-Islander,race,-0.009780,-0.037512,0.168515,0.217291,-8.742759,3.249841,-2.582092,0.030596


In [10]:
from databalanceanalysis.distribution_measures import DistributionBalanceMeasure

# Compute measures
distribution_measures_df = DistributionBalanceMeasure(cols_of_interest).measures(df)

# Transform df into into a dictionary acceptable by the RAI dashboard
rows: List[Any] = distribution_measures_df.reset_index(drop=True).to_dict(orient="records")
distribution_measures: Dict[str, Dict[str, float]] = {
    r["feature_name"]: r for r in rows
}

for feature, measures in distribution_measures.items():
    del measures["feature_name"]

    # TODO: This won't be necessary once cols are converted to strings
    # After that, we can simplify this loop to just .values()
    measures = {k.value: v for k, v in measures.items()}
    distribution_measures[feature] = measures

distribution_measures_df

,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,race,1.055793,0.510400,0.261709,0.654274,0.654274,0.0,87941.889193
1,gender,0.058407,0.121735,0.169205,0.169205,0.169205,0.0,3728.950616


In [11]:
from databalanceanalysis.aggregate_measures import AggregateBalanceMeasure

aggregate_measures_df = AggregateBalanceMeasure(cols_of_interest).measures(df)

aggregate_measures: Dict[str, float] = aggregate_measures_df.reset_index(
    drop=True
).to_dict(orient="records")[0]

# TODO: This won't be necessary once cols are converted to strings
aggregate_measures = {
    k.value: v for k, v in aggregate_measures.items()
}

aggregate_measures_df

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,1.50467,1.120823,0.777909


In [14]:
data_balance_measures: Dict[str, Any] = {
    "distributionBalanceMeasures": {
        "features": list(distribution_measures.keys()),
        "measures": distribution_measures,
    },
    "featureBalanceMeasures": {
        "features": list(feature_measures.keys()),
        "featureValues": unique_values,
        "measures": feature_measures,
    },
    "aggregateBalanceMeasures": aggregate_measures,
}

In [15]:
from responsibleai import DataBalanceAnalysis, RAIInsights
from raiwidgets import ResponsibleAIDashboard

# Pass in information about our data so that RAI can properly compute data balance measures
data_balance_analysis = DataBalanceAnalysis(
    measures=data_balance_measures  # This time, we're passing in pre-computed measures
)

rai_insights = RAIInsights(
    # Params that were already here:
    model=dashboard_pipeline,
    train=train_data_sample,
    test=test_data_sample,
    target_column=target_feature,
    task_type="classification",
    categorical_features=categorical_features,
    # Added params:
    dataset_name="Adult Census",
    data_balance_analysis=data_balance_analysis,
)

ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at http://localhost:5002


### Create Model and Data Insights

In [28]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

To use Responsible AI Toolbox, initialize a RAIInsights object upon which different components can be loaded.

RAIInsights accepts the model, the full dataset, the test dataset, the target feature string, the task type string, and a list of strings of categorical feature names as its arguments.

Add the components of the toolbox that are focused on model assessment.

In [ ]:
from responsibleai import RAIInsights
from raiwidgets import ResponsibleAIDashboard


rai_insights = RAIInsights(
    model=dashboard_pipeline,
    train=train_data_sample,
    test=test_data_sample,
    target_column=target_feature,
    task_type="classification",
    categorical_features=categorical_features,
)

In [31]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the label that they should have, and a list of 
                # strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')

Once all the desired components have been loaded, compute insights on the test set.

In [32]:
rai_insights.compute()

Finally, visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab.

In [33]:
ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at http://localhost:5001


## Assess Your Model

### Aggregate Analysis

The Error Analysis component is displayed at the top of the dashboard widget. To visualize how error is broken down across cohorts, use the tree map view to understand how it filters through the nodes.

![Error Analysis tree map with "Marital Status == 2," "Capital Gain <= 1287.5," "Capital Loss <= 1494.5" path selected](./img/classification-assessment-1.png)

Over 40% of the error in this model is concentrated in datapoints of people who are married, have higher education and minimal capital gain. 

Let's see what else we can discover about this cohort.

First, save the cohort by clicking "Save as a new cohort" on the right side panel of the Error Analysis component.

![Cohort creation sidebar and tree map cohort creation popup](./img/classification-assessment-2.png)

To switch to this cohort for analysis, click "Switch global cohort" and select the recently saved cohort from the dropdown.

![Popup with dropdown to shift cohort from "All data" to "Married, Low Capital Loss/Gain" accompanied by cohort statistics](./img/classification-assessment-3.png)

The Model Overview component allows the comparison of statistics across multiple saved cohorts.

The diagram indicates that the model is misclassifying datapoints of married individuals with low capital gains and high education as lower income (false negative).

![Bar chart of classification outcomes (true negative, true positive, false negative, false positive) compared across cohorts](./img/classification-assessment-4.png)

Looking at the ground truth statistics of the overall data and the erroneous cohort, we realize there are opposite patterns in terms of high income representation in ground truth. While the overall data is representing more individuals with actual income of <= 50K, the married individuals with low capital gains and high education represent more individuals with actual income of > 50K. Given the small size of the dataset and this reverse pattern, the model makes more mistakes in predicting high income individuals. One action item is to collect a lot more data in both cohorts and retrain the model.

![image-3.png](./img/classification-assessment-5.png)

![image.png](./img/classification-assessment-6.png)

The Interpretability component displays feature importances for model predictions at an individual and aggregate level. The plot below indicates that the `marital-status` attribute influence model predictions the most on average.

![Top 5 features of the cohort, in descending importance: relationship, age, capital gain, education-num, hours per week](./img/classification-assessment-7.png)

The lower half of this tab specifies how marita status affects model prediction. Being a husband or wife (married-civ-spouse) is more likely to pull the prediction away from <=50k, possibly because couples have a higher cumulative income.

![Feature importance stratified by relationship](./img/classification-assessment-8.png)

### Individual Analysis

Let's revisit Data Explorer. In the "Individual datapoints" view, we can see the prediction probabilities of each point. Point 510 is one that was just above the threshold to be classified as income of > 50K.

![Scatter plot of prediction probabilities (rounded to 0.2) on the y-axis and index on the x-axis](./img/classification-assessment-9.png)

What factors led the model to make this decision?

The "Individual feature importance" tab in the Interpretability component's Feature Importances section let you select points for further analysis.

![Table of datapoints with row 510 selected](./img/classification-assessment-10.png)

Under this, the feature importance plot shows `capital-gain` and `native-country` as the most significant factors leading to the <= 50K classification. Changing these may cause the threshold to be crossed and the model to predict the opposite class. Please note that depending on the context, the high importance of `native-country` might be considered as a fairness issue.

![Feature importance plot for classification of 0 (descending, positive to negative): age, hours per week, capital gain, race, education-num, workclass, sex, country, occupation, marital status, relationship, capital loss](./img/classification-assessment-11.png)

The What-If Counterfactuals component focuses on how to change features slightly in order to change model predictions. As seen in its top ranked features bar plot, changing this person's marital-status, capital-loss, and education-num have the highest impact on flipping the prediction to > 50K.

![Top-ranked features (descending) for datapoint 510 to perturb to flip model prediction: age, hours per week, capital gain, capital loss, marital status, occupation, education-num, workclass, relationship, race, sex, country](./img/classification-assessment-12.png)